Analyze trip files converted from OMX matrices
1. Are trips sorted by departure time?
2. How does the temporal distribution look?
3. What are the distribution by mode, purpose, VoT?

In [106]:
import numpy as np
import pandas as pd
import h5py
import os
import gc
import time
from tqdm import tqdm_notebook as tqdm

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Execution Parameters

In [131]:
SWIFT_folder = r'C:\Projects\SWIFT\SWIFT_Workspace\Scenarios'
SCEN = 'Scenario_2017_S2_Alpha'
flat_trip_file = os.path.join(SWIFT_folder, SCEN, r'STM\STM_A\01_DynusT\02_Demand\vehicle.csv')

In [132]:
matrix_folder = os.path.join(SWIFT_folder, SCEN, r'STM\STM_D\Outputs_SWIFT')
matrices_am = ["OD AM3HR Vehicles4DynusT EV", "OD AM3HR Vehicles4DynusT NonEV"]
matrices_md = ["OD MD6HR Vehicles4DynusT EV", "OD MD6HR Vehicles4DynusT NonEV"]
matrices_pm = ["OD PM4HR Vehicles4DynusT EV", "OD PM4HR Vehicles4DynusT NonEV"]
matrices_ov = ["OD OV8HR Vehicles4DynusT EV", "OD OV8HR Vehicles4DynusT NonEV"]
matrices = matrices_am + matrices_md + matrices_pm + matrices_ov
matrix_files = [os.path.join(matrix_folder, m+'.omx') for m in matrices]

In [133]:
vots = {
        "wi1da": 9.6, "wi2da": 15.04, "wi3da": 20.48, "wi4da": 27.52, "wi5da": 37.12,
        "wi12a2": 21.56, "wi3a2": 35.84, "wi45a2": 56.56,
        "wi12a3": 30.8, "wi3a3": 51.2, "wi45a3": 80.8,
        "nwi12da": 7.03, "nwi3da": 13.44, "nwi45da": 23.65,
        "nwi12a2": 12.3, "nwi3a2": 23.52, "nwi45a2": 41.39,
        "nwi12a3": 17.57, "nwi3a3": 33.6, "nwi45a3": 59.12,
        "Cargo": 64.0, "Serv": 40.0, "taxi_exta": 18.94, "TNC": 18.94
    }

map_purp = {
    1: "WK", 
    2: "NW", 
    3: "Taxi",
    4: "TNC",
    5: "Serv",
    6: "Cargo",
    
    10: "WK", 
    20: "NW", 
    30: "Taxi",
    40: "TNC",
    50: "Serv",
    60: "Cargo"
    
}

VTYPE_LABEL = {
        1:  'REG_SOV', 
        2:  'REG_HOV',
        3:  'REG_Serv',
        4:  'REG_Cargo',
        5:  'CAV_SOV',
        6:  'CAV_HOV',
        7:  'CAV_Serv',
        8:  'CAV_Cargo',
        9:  'TNC',
        
    }

### Summarize Converted Vehicle Roster

In [134]:
start_time = time.time()

In [135]:
def expand_data(df):
    """
    attach text labels to the data using Categories
    
    """
    VTYPE_LABEL = {
        1:  'REG_SOV', 
        2:  'REG_HOV',
        3:  'REG_Serv',
        4:  'REG_Cargo',
        5:  'CAV_SOV',
        6:  'CAV_HOV',
        7:  'CAV_Serv',
        8:  'CAV_Cargo',
        9:  'TNC',
        
    }
    
    CAV_LABEL = {
        1:  'REG',
        2:  'REG',
        3:  'REG',
        4:  'REG',
        5:  'CAV',
        6:  'CAV',
        7:  'CAV',
        8:  'CAV',
        9:  'REG',
    }

    
    PURP_LABEL = {
        1: 'WK',
        10: 'WK',
        2: 'NW',
        20: 'NW',
        3: 'Taxi',
        30: 'Taxi',
        4: 'TNC',
        40: 'TNC',
        5: 'Serv',
        50: 'Serv',
        6: 'Cargo',
        60: 'Cargo',
        
    }
    
    def period(r):
        r = r / 60.0
        if 0 <= r < 6 or 19 <= r < 24:
            return 'OV'
        elif 6 <= r < 9:
            return 'AM'
        elif 9 <= r < 15:
            return 'MD'
        else:
            return 'PM' 
    
    columns = df.columns.tolist()
    if 'vehtype' in columns:
        df['vehtype2'] = df.vehtype.map(VTYPE_LABEL)
        df['cav'] = df.vehtype.map(CAV_LABEL)
        df['vehtype2'] = df['vehtype2'].astype("category")
        df['cav'] = df['cav'].astype("category")
    if 'TP' in columns:
        df['TP2'] = df.TP.map(PURP_LABEL).astype("category")
    
    if 'stime' in columns:
        df['period'] = df.stime.apply(period).astype("category")
    
    df['ev'] = np.where(df.TP >= 10, 'EV', 'NonEV')
    df['ev'] = df.ev.astype('category')
    

In [136]:
def read_flat_trip_file(flat_trip_file, chunksize=5_000_000):
#     if not write_feather:
#         if feather_trip_file:
#             if not os.path.isfile(feather_trip_file):
#                 print('Error: Feather Trip File not Found {:s}'.format(feather_trip_file))
#                 return
#             else:
#                 df = pd.read_feather(feather_trip_file)
#                 return df

    if not os.path.isfile(flat_trip_file):
        print('Error: Trip File not Found {:s}'.format(flat_trip_file))
        return
    
    # write the feather file in the same directory
#     feather_trip_path, filename = os.path.dirname(flat_trip_file), os.path.splitext(os.path.basename(flat_trip_file))[0] + '.feather'
#     feather_trip_file = os.path.join(feather_trip_path, filename)
    
    dtypes = {
        'vid': np.int32, 
        'usec': np.int32,
        'dsec': np.int32,
        'stime': np.float16,
        'vehcls': np.uint8,
        'vehtype': np.uint8,
        'ioc': np.uint8,
        '#ONode': np.int32,
        '#IntDe': np.uint8,
        'info': np.uint8,
        'ribf': np.float16,
        'comp': np.float16,
        'Izone': np.uint16,
        'Evac': np.uint8,
        'InitPos': np.float32,
        'VoT': np.float16,
        'tFlag': np.uint8,
        'pArrTime': np.float16,
        'TP': np.uint8,
        'IniGas': np.float16,
        'DZone': np.uint16,
        'waitTime': np.float16
    }
    
    usecols = ('vid', 'usec', 'dsec', 'stime', 'vehtype', 'ioc', 'Izone', 'VoT', 'TP', 'DZone')
    
    df_list = []
    
    for df_chunk in tqdm(pd.read_csv(flat_trip_file, usecols=usecols, dtype=dtypes, chunksize=chunksize)):
    
        expand_data(df_chunk)
        df_list.append(df_chunk) 
    
    df = pd.concat(df_list)
    del df_list
    gc.collect()
#     if write_feather:
#         df.to_feather(feather_trip_file)
#         print('Feather file written to {:s}'.format(feather_trip_file))
    return df
    

In [137]:
df_flat_trip_file = read_flat_trip_file(flat_trip_file)

A Jupyter Widget

In [154]:
print('Total Trips for DynusT = {:>,d}'.format(df_flat_trip_file.shape[0]))

Total Trips for DynusT = 18,739,377


In [114]:
# df_flat_trip_file.info()

In [115]:
# df_flat_trip_file.head()

#### <font color='blue'>Summary by all dimensions</font>

In [155]:
summary_roster_gb = df_flat_trip_file.groupby(['period', 'TP2', 'vehtype2', 'cav', 'ev'], as_index=False)['vid'].count().rename(columns={'vid':'trips', 'TP2':'purp'})
summary_roster_gb = summary_roster_gb[~summary_roster_gb.trips.isnull()]

#### <font color='blue'>Summary by Vehicle Type</font>

In [117]:
# summary = pd.DataFrame(df_flat_trip_file.vehtype2.value_counts(sort=False))
# summary = summary.style.format("{:,}")
# summary

### Summarize OMX Trip Tables

In [139]:
def parse_matrix_table(matrix_file_name, matrix_name, vots):
    """
    Parse the matrix name "amnhoai45" for vehicle type, value of time, period, and purpose
    :param matrix_name:
    :type str
    :param vots:        a dict for vots
    :return: a tuple, vehicle_type, occupancy, purpose, value_of_time, period
    """

    matrix_name = matrix_name.lower()

    vtype = occ = purp = vot = 0
    time_period = (0, 0)
    period = ''

    cav, ev = False, False

    if matrix_name.lower().find("da") >= 0:
        if matrix_name.lower().find("cav") >= 0:
            vtype = 5
            occ = 0
            cav = True
        elif matrix_name.lower().find("reg") >= 0:
            vtype = 1
            occ = 0
            cav = False

    elif matrix_name.lower().find("s2") >= 0 or matrix_name.find("a2") >= 0:
        if matrix_name.lower().find("cav") >= 0:
            vtype = 6
            occ = 1
            cav = True
        elif matrix_name.lower().find("reg") >= 0:
            vtype = 2
            occ = 1
            cav = False
    elif matrix_name.lower().find("s3") >= 0 or matrix_name.find("a3") >= 0:
        if matrix_name.lower().find("cav") >= 0:
            vtype = 6
            occ = 1
            cav = True
        elif matrix_name.lower().find("reg") >= 0:
            vtype = 2
            occ = 1
            cav = False
    elif matrix_name.lower().find("cargo") >= 0:
        if matrix_name.lower().find("cav") >= 0:
            vtype = 8
            occ = 2
            cav = True
        else:
            vtype = 4
            occ = 2
            cav = False
    elif matrix_name.lower().find("serv") >= 0:
        if matrix_name.lower().find("cav") >= 0:
            vtype = 7
            occ = 2
            cav = True
        else:
            vtype = 3
            occ = 2
            cav = False
    elif matrix_name.lower().find("taxi") >= 0:
        vtype = 2
        occ = 1
    elif matrix_name.lower().find("exta") >= 0:
        vtype = 2
        occ = 0
    elif matrix_name.lower().find("tnc") >= 0:
        vtype = 9
        occ = 1

    if matrix_name.find("_w") >= 0:
        if matrix_file_name.lower().find("dynust nonev") >= 0:
            purp = 1
        else:
            purp = 10
            ev = True
    elif matrix_name.find("_nw") >= 0:
        if matrix_file_name.lower().find("dynust nonev") >= 0:
            purp = 2
        else:
            purp = 20
            ev = True
    elif matrix_name.lower().find("taxi") >= 0:
        if matrix_file_name.lower().find("dynust nonev") >= 0:
            purp = 3
        else:
            purp = 30
            ev = True
    elif matrix_name.lower().find("tnc") >= 0:
        if matrix_file_name.lower().find("dynust nonev") >= 0:
            purp = 4
        else:
            purp = 40
            ev = True
    elif matrix_name.lower().find("serv") >= 0:
        if matrix_file_name.lower().find("dynust nonev") >= 0:
            purp = 5
        else:
            purp = 50
            ev = True
    elif matrix_name.lower().find("cargo") >= 0:
        if matrix_file_name.lower().find("dynust nonev") >= 0:
            purp = 6
        else:
            purp = 60
            ev = True

    # Time periods
    if matrix_file_name.lower().find("am") >= 0:
        period = 'AM'
        time_period = '6..9'
    elif matrix_file_name.lower().find("md") >= 0:
        period = 'MD'
        time_period = '9..15'
    elif matrix_file_name.lower().find("pm") >= 0:
        period = 'PM'
        time_period = '15..19'
    else:
        period = 'OV'
        time_period = '0..6, 19..24'

    for key in vots.keys():
        if matrix_name.find(key) >= 0:
            vot = vots[key]
            break

    return dict(vtype=vtype, occ=occ, purp=purp, vot=vot, time_period=time_period, period=period, cav=cav, ev=ev)

In [140]:
def summarize_trip_tables(matrix_files, vots):
    summary_records = []
    for m in matrix_files:
#         print('Processing Matrix File {:s}'.format(m))
        h5 = h5py.File(m, 'r')
        tables = h5['/data/'].keys()
        for table in tqdm(tables, desc=os.path.split(m)[1].split('.')[0]):
            if table.find('all') >= 0:
                continue
            info = parse_matrix_table(m, table, vots)
            vtype = info['vtype']
            occ = info['occ']
            purp = info['purp']
            vot = info['vot']
            period = info['period']
            cav = info['cav']
            ev = info['ev']
            
            if cav:
                cav = 'CAV'
            else:
                cav = 'REG'
            
            if ev:
                ev = 'EV'
            else:
                ev = 'NonEV'
            
            od = h5['/data/' + table][:]
            trips = od.sum()
            summary_records.append((table.lower(), period, purp, vot, vtype, occ, cav, ev, trips))
        h5.close()
    return pd.DataFrame(data=summary_records, columns=['table', 'period', 'purp', 'vot', 'vtype', 'occ', 'cav', 'ev', 'trips'])
    

In [141]:
summary_omx = summarize_trip_tables(matrix_files, vots)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [142]:
summary_omx['purp'] = summary_omx.purp.map(map_purp)
summary_omx['vehtype2'] = summary_omx.vtype.map(VTYPE_LABEL)

In [143]:
summary_omx_gb = summary_omx.groupby(['period', 'purp', 'vehtype2', 'cav', 'ev'], as_index=False)['trips'].sum()
summary_omx_gb = summary_omx_gb[summary_omx_gb.trips > 0]

In [151]:
summary_omx_gb

,period,purp,vehtype2,cav,ev,trips
1,AM,Cargo,REG_Cargo,REG,NonEV,24588.860
7,AM,NW,REG_HOV,REG,NonEV,416778.647
9,AM,NW,REG_SOV,REG,NonEV,1200798.044
11,AM,Serv,REG_Serv,REG,NonEV,99252.011
13,AM,TNC,TNC,REG,NonEV,1824.877
15,AM,Taxi,REG_HOV,REG,NonEV,102211.693
21,AM,WK,REG_HOV,REG,NonEV,93850.513
23,AM,WK,REG_SOV,REG,NonEV,1117931.476
25,MD,Cargo,REG_Cargo,REG,NonEV,63157.657
31,MD,NW,REG_HOV,REG,NonEV,1214770.291


#### Compare Vehicle Roster to OMX

In [157]:
join_cols = ['period', 'purp', 'vehtype2', 'cav', 'ev']
summary_comparison = pd.merge(left=summary_omx_gb, right=summary_roster_gb, on=join_cols, suffixes=('_OMX', '_DY'),)
summary_comparison['Abs_Diff'] = summary_comparison.trips_DY - summary_comparison.trips_OMX
summary_comparison['Rel_Diff'] = summary_comparison.Abs_Diff / summary_comparison.trips_OMX * 100.0
summary_comparison_show = summary_comparison.style.format({'trips_OMX': "{:,.2f}", 'trips_DY': '{:,.0f}', 'Rel_Diff': '{:.2f}%'})
summary_comparison_show

,period,purp,vehtype2,cav,ev,trips_OMX,trips_DY,Abs_Diff,Rel_Diff
0,AM,Cargo,REG_Cargo,REG,NonEV,"24,588.86","24,596",7.14,0.03%
1,AM,NW,REG_HOV,REG,NonEV,"416,778.65","416,880",101.353,0.02%
2,AM,NW,REG_SOV,REG,NonEV,"1,200,798.04","1,201,126",327.956,0.03%
3,AM,Serv,REG_Serv,REG,NonEV,"99,252.01","99,306",53.9887,0.05%
4,AM,TNC,TNC,REG,NonEV,"1,824.88","1,823",-1.8773,-0.10%
5,AM,Taxi,REG_HOV,REG,NonEV,"102,211.69","102,218",6.3073,0.01%
6,AM,WK,REG_HOV,REG,NonEV,"93,850.51","93,814",-36.5132,-0.04%
7,AM,WK,REG_SOV,REG,NonEV,"1,117,931.48","1,117,531",-400.476,-0.04%
8,MD,Cargo,REG_Cargo,REG,NonEV,"63,157.66","63,118",-39.6575,-0.06%
9,MD,NW,REG_HOV,REG,NonEV,"1,214,770.29","1,213,936",-834.291,-0.07%


In [158]:
total_OMX, total_DY = sum(summary_comparison.trips_OMX), sum(summary_comparison.trips_DY)
print('Total Trips for OMX    = {:>,.0f}'.format(total_OMX))
print('Total Trips for DynusT = {:>,.0f}'.format(total_DY))
print('Total Abs. Difference  = {:>,.0f}'.format(total_DY-total_OMX))
print('Total Rel. Difference  = {:>.2f}%'.format((total_DY-total_OMX)/total_OMX*100.0))

Total Trips for OMX    = 18,739,379
Total Trips for DynusT = 18,739,377
Total Abs. Difference  = -2
Total Rel. Difference  = -0.00%


In [159]:
summary_comparison.to_csv(os.path.join(os.path.dirname(flat_trip_file), 'summary_comparison.csv'), index=False)

In [160]:
end_time = time.time()
execution_time = (end_time - start_time) / 60.0
print("ALL DONE in {:.0f} minutes".format(execution_time))

ALL DONE in 14 minutes


In [161]:
import itertools
periods = ('AM', 'MD', 'PM', 'OV')
purposes = ('WK', 'NW', 'Cargo', 'Serv', 'Taxi', 'TNC')
CAV = ('REG', 'CAV')
EV = ('EV', 'NonEV')
pd.DataFrame(list(itertools.product(periods, purposes, CAV, EV))).to_csv(os.path.join(os.path.dirname(flat_trip_file), 'cat.csv'))